# OpenStreetMap Project - Chicago

This project will use the map of a beautiful city, Chicago, IL, United States. I have lived here since graduating from college. I am very interested to see what the map database reveals. After unziping, the total database is a little more than 2GB.

I will analyze this dataset by doing the following:

* Extract a sample from the database.
* Find the problems encountered in this dataset. 
* Clean up the data and import them to SQL.
* Explore the data by querying in SQLite.
* Additional ideas I have after exploring the dataset.

Reference:

* The summary of Chicago area can be found at [OpenStreetMap website](https://www.openstreetmap.org/relation/122604). 
* This data can be downloaded at [Mapzen Metro Extracts](https://mapzen.com/data/metro-extracts/metro/chicago_illinois/). 
* [OpenStreetMap Wiki](https://wiki.openstreetmap.org/wiki/Main_Page) shows the detail explanation of OpenStreetMap database.

## Import Libraries

In [1]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import lxml
import cerberus
from collections import defaultdict
import sqlite3

## Extract a sample

As mentioned before, this database is quite large, more than 2GB. Directly opening it or parsing it will crash the computer. Therefore, it is a good idea to extract a sample from this dataset. 

I will use the extract-sample-data.py file to extract 1% of the original data. This only needs to run once. The final sample file is around 20MB.

In [2]:
%%timeit -r 1
%run extract-sample-data.py

1 loop, best of 1: 2min 20s per loop


After getting the sample from the database, it is a good idea to see the big picture of this sample to see if we have had enough data within the sample. Therefore, I want to write a function to check what tags are in the sample dataset, and how many of them.

In [2]:
sample_file = 'sample_chicago.osm'

In [3]:
def count_tag(filename):
    tags = {}
    for event, elem in ET.iterparse(filename):
        tag = elem.tag
        if tag not in tags:
            tags[tag] = 1
        else:
            tags[tag] += 1
    return tags

In [5]:
count_tag(sample_file)

{'member': 349,
 'nd': 103077,
 'node': 87172,
 'osm': 1,
 'relation': 48,
 'tag': 67876,
 'way': 12337}

It seems to be that we have a good amount of data within the sample. 

## Problem in this dataset

After getting the sample data, we can look through the dataset, find the problems and clean it up.

Through reading the documente and look through the sample data in a text editor, `<tag>` is used to save all the values. 

Here are some problems I noticed the following potential problems through reading the sample data:

* The `<tag>`'s k attribute value is not consistent. Some only have lower case like "ele". Some have both lower case and colon, like "gnis: id". Others have special characters like.
* The street name is not consistent. Some uses the whole spell, like "street" and "avenue", while others use abbreviation, like "Ave".
* The phone number format is not consistent. Some have (XXX) XXX-XXXX while others have XXX-XXX-XXXX.

### k attribute issue

The k-attribute has three main patterns:

* The k-attribute values only contain lowercase letter, i.e. "building".
* The k-attribute values contains both lowercase letter and colon, i.e. "addr: city".
* The problematic pattern will contains special characters like "&".'
* The rest will be "others".

The first two patterns and "others" are good. They will not influence future analysis. However, the third one needs some clean-up. I will run the k_attrib_type.py file to find the patterns within my sample file.

In [6]:
%run "k-attribute-issues.py"

k_attrib_type(sample_file, keys)

{'lower': 20677, 'lower_colon': 31308, 'other': 15891, 'problemchars': 0}

Based on this result, there is no problematic characters within k attributes. Therefore, we do not need to clean k attribute for future analysis.

### v attribute issue

The v-attribute contains the value for k-attribute. There are two v attributes that I found have some potential problems after looking through the sample file in a text editor.

* Many of the street name in this file use abbreviation. For example, it uses 'Dr' instead of 'Drive'. It may causes problems in later analysis. Therefore, I need to find abbreviation and fix them.
* The phone number in this .osm file is not consistent. After looking through a small sample of this file, I found at least four kinds of format. Some phone numbers look like "XXX-XXX-XXXX", some look like "(XXX) XXX-XXXX", some look like "+1-XXX-XXX-XXXX" while others look like "(XXX)XXX-XXXX". There might be other formats as well. 

In [7]:
%run audit-v-value.py

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
expected = ['Street', 'Avenue', 'Boulevard', 'Drive', 'Court', 'Place',
            'Square', 'Lane', 'Road', 'Trail', 'Parkway', 'Commons', 'Broadway', 'Circle'
           'Park', 'Path', 'Terrace', 'West', 'Highway']

audit(sample_file, "addr:street", street_type_re, expected)

defaultdict(set,
            {'14': {'U.S. 14'},
             'Ave': {'Alabama Ave', 'New York Ave'},
             'Ave.': {'Ogden Ave.'},
             'B': {'South Avenue B'},
             'C': {'South Avenue C'},
             'C405': {'S Williams St #C405'},
             'Circle': {'Woodland Park Circle'},
             'Ct': {'Boulder Ct', 'Timber Ct', 'Vail Ct'},
             'Dr': {'Breckenridge Dr',
              'Greenbriar Dr',
              'Gregory M Sears Dr',
              'John M Boor Dr',
              'Summit Dr'},
             'E': {'South Avenue E'},
             'F': {'South Avenue F'},
             'G': {'South Avenue G'},
             'H': {'South Avenue H'},
             'J': {'South Avenue J'},
             'L': {'South Avenue L'},
             'Ln': {'Leadville Ln'},
             'M': {'South Avenue M'},
             'N': {'900 N', 'South Avenue N'},
             'O': {'South Avenue O'},
             'Park': {'West Midway Park'},
             'St': {'Kathleen St',

In [8]:
phone_type_re = re.compile(r'(\+1-)?\(?\d\d\d\)?[-| ]?\d\d\d[-| ]?\d\d\d\d')
expected = re.compile(r'^\d\d\d-\d\d\d-\d\d\d\d$')

audit(sample_file, "phone", phone_type_re, expected)

defaultdict(set,
            {'(312) 369-7900': {'(312) 369-7900'},
             '(708) 749-0895': {'(708) 749-0895'},
             '(847)434-0300': {'(847)434-0300'},
             '(847)806-1230': {'(847)806-1230'},
             '+1-708-715-7746': {'+1-708-715-7746'},
             '219-988-2111': {'+1 219-988-2111'}})

## Generate the csv files

We have audited the Chicago osm file, and it is time to clean it and generate the csv files we need.

Based on the previous analysis, there is no problematic characters within 'k' attributes. I will not update this part.

The street type is inconsistent. There are many abbreviations inside. I will creating a mapping to update these parts.

There are only six phone numbers that have format issues. It is too little compared to the whole datasets. Therefore, I will not clean them.

In [18]:
%run generate-csv-file.py

process_map(sample_file, validate=True)

## Create the database

After generating the csv files we need, it is time to transfer the csv files into a sqlite database. The shcema of this database is stored in schema.py file.

In [55]:
%run create-database.py

## Data Overview

After creating the database, we can start the query and find some interesting facts within this database. Let's do some simple query to get an overview of this dataset.

In [2]:
conn = sqlite3.connect('osm_chicago.db')
c = conn.cursor()

### Number of nodes

In [3]:
Q = '''
    SELECT COUNT(*) FROM nodes;
'''

c.execute(Q).fetchall()

[(87172,)]

### Number of ways

In [4]:
Q = '''
    SELECT COUNT(*) FROM ways;
'''

c.execute(Q).fetchall()

[(12337,)]

### Number of unique users

In [9]:
Q = '''
    SELECT COUNT (DISTINCT(total.uid)) FROM
        (SELECT uid FROM nodes
        UNION ALL SELECT uid FROM ways) total;
'''

c.execute(Q).fetchall()

[(947,)]

## Detail Exploration

I am a big foodie, so I am very interested in the restaurant in Chicago. First I want to know how many restaurants per cuisince type in the sample .osm file I generated.

In [20]:
def cuisine_number(cursor):
    '''
    This function will return the # of restaurants of each cuisine types
    within the osm database
    '''
    Q = '''
    SELECT value, COUNT (DISTINCT(id))
    FROM nodes_tags
    WHERE key = 'cuisine'
    GROUP BY value
    ORDER BY 2 DESC
    '''

    return cursor.execute(Q).fetchall()

In [21]:
len(cuisine_number(c))

15

It actually looks a little big weird. Based on my experience, Chicago has way more restaurants than this. Let's check the total number of restaurant

In [22]:
def num_restaurant(cursor):
    '''
    This function will return the total number of restaurants
    within the database
    '''
    QUERY = '''
    SELECT COUNT (DISTINCT(id)) FROM nodes_tags
    WHERE nodes_tags.value = 'restaurant';
    '''

    results = cursor.execute(QUERY).fetchall()
    return results

In [23]:
num_restaurant(c)

[(17,)]

Based on the analysis above, not all the restaurants in the map has showed their cuisine types. Therefore, the two amounts are not the same. But I still doubt about the total number of restaurants. Let's do a quick analysis of the postcode for each cuisine type.

In [24]:
def cuisine_location(cursor, cuisine_type):
    '''
    This function will return the postcode and the cuisine type
    '''
    QUERY = '''
    SELECT nodes_tags.value, COUNT(*)
    FROM nodes_tags
    JOIN
        (SELECT DISTINCT (id) FROM nodes_tags
        WHERE key = 'cuisine' AND value = "{}") as second_nodes_tags
    ON nodes_tags.id = second_nodes_tags.id
    WHERE nodes_tags.key = 'postcode'
    GROUP BY 1
    ORDER BY 2 DESC;
    '''.format(cuisine_type)

    results = cursor.execute(QUERY).fetchall()
    return results

In [25]:
cuisine_location(c, 'japanese')

[(u'60616', 1)]

In [26]:
cuisine_location(c, 'mexican')

[]

Mexican cuisine shows zero, but it shows 1 in the above analysis. This could mean the postcode is missing for the mexican restaurants. Let's check it out.

In [27]:
def cuisine_wo_code(cursor, cuisine_type):
    '''
    This function will return the id and # of restaurants that
    don't have a postcode.
    '''
    Q = '''
    SELECT nodes_tags.id, nodes_tags.value
    FROM nodes_tags
    WHERE (id NOT IN
        (SELECT DISTINCT(id) FROM nodes_tags
        WHERE key = 'postcode'))
    AND (key = 'cuisine')
    AND (value = "{}")
    '''.format(cuisine_type)

    results = cursor.execute(Q).fetchall()
    return results

In [28]:
cuisine_wo_code(c, 'mexican')

[(1317827223, u'mexican')]

## Conclusion & Additional Ideas

Based on the auditing process above, I think this is a very cleaned datasets for the purposes of this exercise. A lot of people has made a contribution to achieve it. There are still some improvements can be made.

* The phone number doesn't have a consistent format. Based on the [phone number formatting guide](http://stdcxx.apache.org/doc/stdlibug/26-1.html), since osm is for mapping information around the world, '+1-XXX-XXX-XXXX' can be a good format for US number. The website can put some formatting validation or corrections for better formatting.
* As I mentioned above, this dataset may be incomplete. There is only 17 restaurants within 1% of the total dataset. The sample data is relatively randome, so it shows the whole dataset does not have enough restaurants. It might because not many restaurant owners input their data into the dataset. To better improve the completeness of dataset, there can be some incentives for users to keep contributing to this dataset. For example, getting coupons of a restaurant after gathering the information from that restaurant. :)

There are many benefits as well as some potential issues related to the two improvements.

Make the phone number format consistent
* Benefits:
    * Based on the recommended formatting above, the phone number can show the country code. People can easily see which country does the 'node' belongs to, and can also point out if the country code is correct.
    * People can also see the total number of digits. Therefore, problems with wrong number of digits can be avioded. 
* Potential Issues:
    * There are chances that people do not know their country code when inputing the information. They may end up not writing down the number at all.
    * The formatteing suggested are based on US number. Some phone number format, like Chinese phone number, will have a totally different format.
    
Improve the completeness of the map
* Benefits:
    * For a mapping data, completeness can be the most important thing. For a foodie like me, restaurants information is the first thing I want to learn. Incomplete information could frustrate many end-users.
* Potential Issues:
    * The cost to get all the data could be very high. People may not have the incentive to gather all the information needed.
    * It is very hard to measure completeness. "You don't know what you don't know". Therefore, we need to set a standard first before diving in the gathering process. Otherwise, it can be a infinit process.